<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/20_probability/20_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 간단한 통계 기능 소개<br>Simple introduction to statistics features



## `pandas`



`pandas`의 통계 기능에 대해 알아보자.<br>Let's check the statistics features of the `pandas`



In [ ]:
# NumPy & matplotlib
import pylab as py

# Data table
import pandas as pd



데이터 배열 생성<br>Creating data arrays



In [ ]:
t_deg = py.linspace(-360, 360, 24+1)
t_rad = py.deg2rad(t_deg)
sin_t = py.sin(t_rad)
cos_t = py.cos(t_rad)



데이터 표 생성<br>Creating data table



In [ ]:
df = pd.DataFrame(
    {
        't_rad': t_rad,
        'sin': sin_t,
        'cos': cos_t,
    },
    index=t_deg,
    columns=['t_rad', 'sin', 'cos']
)



데이터 표 내용<br>Content of the data table



In [ ]:
# https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
# set maximum number of rows to display
pd.options.display.max_rows = 10
df



데이터 표 정보<br>Data table info



In [ ]:
print(f'df.shape = {df.shape}')
print(f'df.columns = {df.columns}')



이름으로 열 선택<br>Selecting a column by its name



In [ ]:
print(f'df["sin"] = \n{df["sin"]}')



논리식으로 행 선택<br>Choosing rows by a boolean logic



In [ ]:
print(f"df[abs(df.sin)<1e-3] = \n{df[abs(df.sin)<1e-3]}")



다양한 통계<br>Various statistics



In [ ]:
df.describe()



산포도 행렬<br>Scatter matrix



In [ ]:
import pandas.plotting as plotting
plotting.scatter_matrix(df[['t_rad', 'cos','sin']])



## 회귀 분석<br>Regression Analysis



회귀분석이란 예를 들어 $x$ 와 $y$ 두 변수 사이의 관계를 통계적인 방법으로 탐색하는 것이다.<br>
Regression analysis is to search for a relationship statistically, for example, between $x$ and $y$. [[wikipedia](https://en.wikipedia.org/wiki/Regression_analysis)]



[![Introduction to residuals and least squares regression](https://img.youtube.com/vi/yMgFHbjbAW8/0.jpg)](https://youtu.be/yMgFHbjbAW8)



### 데이터 준비<br>Prepare data



참값이 아래에서 구한 값과 같았다고 가정하자.<br>
Let's assume that following cell generates the true value.



In [ ]:
import pylab as py

a = 0.5
b = 2.0

x_array = py.linspace(0, 5, 20 + 1)
y_true = a * x_array + b



방금 구한 참값을 그림으로 표시해 보자.<br>
Let's plot the true values that we just generated.



In [ ]:
py.plot(x_array, y_true, '.-', label='true')
py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



잡음이 섞인 측정값도 준비해 보자.<br>
Lets' prepare for the measurements contaminated by some noise.



In [ ]:
import numpy.random as nr

nr.seed()

w_array = nr.normal(0, 0.25, size=x_array.shape)
y_measurement = y_true + w_array



이것도 그려 보자.<br>
Let's plot this too.



In [ ]:
py.plot(x_array, y_true, label='true')
py.plot(x_array, y_measurement, '.', label='measurements')

py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



### 선형회귀와 추정<br>Linear Regression and Estimation



$x$와 $y$사이의 관계가 다음과 같았다고 가정해 보자.<br>
Let's assume that $x$ and $y$ have following relationship.



$$
y = ax + b
$$



$a$와 $b$가 어떤 값을 가지면 위에서 구한 데이터와 비교해 볼 때 가장 적합하겠는가?<br>How can we find $a$ and $b$ fitting the curve the best against the data above?



이러한 탐색 과정을 *선형회귀*라고 부를 수 있을 것이다.<br>We may call this search process as *Linear Regression*.



선형 회귀 결과는 다음과 같다<br>The results from the linear regression is as follows.<br>
ref : The SciPy community, "scipy.stats.linregress", SciPy documentation, May 05, 2018 [Online] Available : https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html.



In [ ]:
import scipy.stats as ss

slope, intercept, r_value, p_value, std_err = ss.linregress(x_array, y_measurement)
print(f'slope = {slope}')
print(f'intercept = {intercept}')
print(f'correlation coefficient = {r_value}')



이 결과를 이용하여 $y$값을 추정해 보자.<br>
Let's estimate $y$ using this result.



$$
y_{estim} = slope \cdot x_{array} + intercept
$$



In [ ]:
def linear_model(a, b, x_i):
    return a * x_i + b

y_lin_reg = linear_model(slope, intercept, x_array)



이 결과를 그려보자.<br>
Let's plot this result.



In [ ]:
py.plot(x_array, y_true, label='true', alpha=0.3)
py.plot(x_array, y_measurement, 'o', label='measurements')
py.plot(x_array, y_lin_reg, '.-', label='lin reg')

py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



### 다항식 회귀 사례<br>Polynomial Regression Example



이번에는 $x$와 $y$사이의 관계가 다음과 같았은 다항식의 형태를 가진다고 가정해 보자.<br>
This time, let's assume that $x$ and $y$ are related in the following polynomial form.



$$
y = ax^2 + bx + c
$$



데이터를 위한 최적의 곡선을 찾기 위해 $b$와 $c$에 더하여 $a$도 바꾸어 볼 수 있다.<br>
Now we can tweak $a$ in addition to $b$ and $c$ to fit the curve to the data.



references :

* Ahush Pant, "Introduction to Linear Regression and Polynomial Regression", Towards Data Science, Medium, Jan 13, 2019, [Online](https://towardsdatascience.com/introduction-to-linear-regression-and-polynomial-regression-f8adc96f31cb).
* Nikolay Mayorov, "Robust nonlinear regressio in scipy", Scipy Cookbook, Aug 17, 2018, [Online](https://scipy-cookbook.readthedocs.io/items/robust_regression.html).



`scipy.optimize` 의 `leastsq()`를 사용할 것이므로 해당 모듈을 읽어들인다.<br>
We are going to use `leastsq()` of `scipy.optimize`.



In [ ]:
import scipy.optimize as so



`scipy.optmize.leastsq()` 에 대해서는 아래 셀에서 `#`를 지우고 <kbd>Shift</kbd>+<kbd>Enter</kbd>를 눌러 본다.<br>
Regarding `scipy.optmize.leastsq()`, delete `#` in the following cell and press <kbd>Shift</kbd>+<kbd>Enter</kbd>.



In [ ]:
# help(so.leastsq)



계수를 매개변수로 받아들이는 2차 다항식 모델을 함수로 구현한다.<br>
Accepting coefficients from parameters, let's implement a function of a second order polynomial.



In [ ]:
def polynomial_model_2(param, x_i):
    a, b, c = param

    return a * x_i ** 2 + b * x_i + c



각 점에서 측정값과 추정값 사이의 오차를 계산하는 함수를 구현한다.<br>
Implement another function calculating the error between estimation and measurement at each data point.



In [ ]:
def polynomial_error(param, x_i, y_i, model=polynomial_model_2):
    y_i_estimation = model(param, x_i)

    error_array = y_i_estimation - y_i

    return error_array



다항식의 계수를 회귀로 추정한 결과는 다음과 같다.<br>
The following cell estimates the coefficients of the polynomial using the regression.



In [ ]:
any_initial_guess = (1, 1, 1)

polynomial_regression_param = so.leastsq(
    polynomial_error, 
    any_initial_guess, 
    args=(x_array, y_measurement)
)

polynomial_regression_param



해당 계수는 예를 들어 다음과 같이 사용할 수 있을 것이다.<br>
We could use the coefficients as follows.



In [ ]:
a_reg, b_reg, c_reg = polynomial_regression_param[0]

y_poly_reg = a_reg * x_array ** 2 + b_reg * x_array + c_reg



이 결과를 그려보자.<br>
Let's plot this result.



In [ ]:
py.plot(x_array, y_true, label='true', alpha=0.3)
py.plot(x_array, y_measurement, 'o', label='measurements')
py.plot(x_array, y_lin_reg, '.-', label='lin reg')
py.plot(x_array, y_poly_reg, '.', label='poly reg 2')

py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



### 과적합<br>Overfitting



이제 좀 더 일반적인 경우를 생각해 보자.<br>Let's think about a more general case.



다항식의 최고 차수가 2차 대신 $n$차인 경우를 생각해 보자.<br>What if the highest order of the polynomial is $n$ instead of two?



In [ ]:
def polynomial_model_n(param, x_i):
    return py.polyval(param, x_i)



In [ ]:
n_reg = 10
any_initial_guess = py.ones((n_reg,))

polynomial_regression_param_n = so.leastsq(
    polynomial_error, 
    any_initial_guess, 
    args=(x_array, y_measurement, polynomial_model_n)
)

polynomial_regression_param_n



In [ ]:
y_poly_reg_n = polynomial_model_n(polynomial_regression_param_n[0], x_array)



이 결과를 그려보자.<br>
Let's plot this result.



In [ ]:
py.plot(x_array, y_true, label='true', alpha=0.3)
py.plot(x_array, y_measurement, 'o', label='measurements')
py.plot(x_array, y_lin_reg, '.', label='lin reg')
py.plot(x_array, y_poly_reg, '.', label='poly reg 2')
py.plot(x_array, y_poly_reg_n, 'x', label='poly reg n')


py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



측정값 사이는 어떨까?<br>
What about between the measurements?



In [ ]:
x_min, x_max = x_array.min(), x_array.max()
x_array2 = py.linspace(x_min - 0.25, x_max + 0.25)

py.plot(x_array2, a * x_array2 + b, label='true', alpha=0.3)
py.plot(x_array, y_measurement, 'o', label='measurements')

x_detailed = py.linspace(x_array2.min(), x_array2.max(), len(x_array) * 10)

py.plot(x_detailed, linear_model(slope, intercept, x_detailed), '-', label='lin reg')
py.plot(x_detailed, polynomial_model_2(polynomial_regression_param[0], x_detailed), '-', label='poly reg 2')
py.plot(x_detailed, polynomial_model_n(polynomial_regression_param_n[0], x_detailed), '-', label='poly reg n')


py.grid(True)
py.ylim(ymin=0)
py.legend(loc=0)
py.xlabel('x')
py.ylabel('y');



## 연습 문제<br>Exercises



도전 과제 1: 위 선형 회귀에 사용되었던 자료를 판다스 데이터프레임으로 저장하고 다양한 통계값을 계산해 보시오.<br>Try this 1: Store the data for linear regression in a `pandas.DataFrame` and calculate various statistics.



도전 과제 2: 공신력 있는 기관에서 공개한 변수가 2개 이상인 자료를 찾아 도전 과제 1을 반복하시오.<br>Try this 2: Find data having more than two variables from a credible organization and repeat Try this 1 above.



도전 과제 3: 도전 과제 2의 자료에 대해 선형회귀를 적용해 보시오.<br>Try this 3: Apply linear regerssion to data of Try this 2.



도전 과제 4: 선형회귀의 수학적 원리를 설명해 보시오.<br>Try this 4: Describe mathematic of the linear regression



도전 과제 5: 도전과제 4 를 구현하여 위 사이파이 선형회귀 결과와 비교해 보시오.<br>Try this 5: Implement Try this 4 and compare with the linear regression result using SciPy.



## `alpha`



`alpha`로 그래프의 투명도를 선택할 수 있다.<br>With `alpha`, we can control the plots' transparency.



In [ ]:
import pylab as py


x = py.linspace(0, 1)

for alpha_value in py.arange(1, 0-0.01, -0.1):
    y = alpha_value * x
    py.plot(x, y, alpha=alpha_value, label=f"$\\alpha$={alpha_value:3.1f}")

py.legend(loc=0);



## 소프트웨어 시험 함수<br>Software Test Functions



아래는 위 함수가 맞게 작성되었는지 확인한다.<br>Followings test functions above.



In [ ]:
def test_polynomial_model_2():

    param = (1, -3, 2)
    x_test = py.array([0, 1, 2])

    y_result = polynomial_model_2(param, x_test)
    y_expected = py.polyval(param, x_test)

    assert y_result.shape == y_expected.shape, f"\ny_result = {y_result}\n!= y_expected = {y_expected}"



In [ ]:
test_polynomial_model_2()



In [ ]:
def test_square_error_sum_true():

    param = (1, -3, 2)
    x_test = py.array([0, 1, 2])
    y_test_true = py.array([2, 0, 0])

    y_result = polynomial_error(param, x_test, y_test_true)
    y_expected = py.polyval(param, x_test) - y_test_true

    assert y_result.tolist() == y_expected.tolist(), f"\ny_result = {y_result}\n!= y_expected = {y_expected}"



In [ ]:
test_square_error_sum_true()



In [ ]:
def test_square_error_sum_not_true():

    param = (1, -3, 2)
    x_test = py.array([0, 1, 2])
    y_test_not_true = py.array([1, 1, 1])

    y_result = polynomial_error(param, x_test, y_test_not_true)
    y_expected = py.polyval(param, x_test) - y_test_not_true

    assert y_result.tolist() == y_expected.tolist(), f"\ny_result = {y_result}\n!= y_expected = {y_expected}"



In [ ]:
test_square_error_sum_not_true()



## 참고문헌<br>References



* 맥키니 저, 김영근 역, 파이썬 라이브러리를 활용한 데이터 분석, 2판, 한빛미디어, 2019, ISBN 979-11-6224-190-5 ([코드와 데이터](https://github.com/wesm/pydata-book/)) <br>Wes McKinney, Python for Data Analysis, 2nd Ed., O'Reilly, 2017. ([Code and data](https://github.com/wesm/pydata-book/))
* Varoquaux, Statistics in Python, Scipy lecture notes, 2018 Sept 01, [Online] Available: http://www.scipy-lectures.org/packages/statistics/index.html.



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

